# Modelagem de dados em Python 🐍



Este tutorial mostra como construir alguns tipos de gráficos, testes t-Student e tabelas de contingência.

O primeiro passo, como antes, é carregar os dados.

In [ ]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('mpg')
df['cylinders'] = df['cylinders'].astype('category')
df.head()

## Comparando o consumo pelo número de cilindros

A biblioteca `Seaborn` permite criar *boxplot*s com dados em ambos os eixos. Para o *boxplot* fazer algum sentido, a variável `x` precisa ser numérica e a variável `y` é considerada categórica. Assim, obtém-se a comparação entre consumo e número de cilindros.

In [ ]:
#ensure Seaborn is loaded
import seaborn as sns

sns.boxplot(x=df['cylinders'], y=df['mpg'], showmeans=True);

### Histogramas

É possível construir histogramas para cada classe de uma variável categórica. Isto é feito com os argumentos `row` ou `col` no comando `distplot()`:

In [ ]:
sns.displot(x='mpg', col='cylinders', data=df, linewidth=0, kde=True);

## Sobrepondo gráficos de kernel de densidades

Sobrepor histogramas é uma estratégia ruim, uma vez que o gráfico fica confuso:

In [ ]:
sns.histplot(x='mpg', hue='cylinders', data=df, linewidth=0);

Uma estratégia melhor para comparar distribuições é sobrepor gráficos de *kernel* das respectivas distribuições.

In [ ]:
sns.kdeplot(x='mpg', hue='cylinders', data=df, fill=True);

## Teste t-Student

Os testes **t-Student** são bastante conhecidos e usados para a comparação de médias entre grupos quando a suposição de que os dados são provenientes de uma distribuição Normal é razoável. O teste pode ser realizado comparando 2 vetores numéricos.

### Formatando os dados

A forma mais fácil de realizar um teste t-Student no *Python* é separando os vetores de acordo com os grupos desejados. Por exemplo, supondo que se deseja comparar o consumo dos veículos com motor de 4 cilindros contra o consumo dos veículos com mais que 4 cilindros:

In [ ]:
cyl4 = df[df['cylinders'] == 4]['mpg']
cylm = df[df['cylinders'] != 4]['mpg']

### Testando a igualdade de variância (entre grupos)

Por padrão o *R* usa o **teste F** para comparar variâncias. A biblioteca *Scipy* não implementa o teste F, mas ao invés implementam o **teste de Levene** e o **teste de Bartlett** (os testes de Levene e de Bartlett também estão disponíveis no *R*). Por exemplo, o teste de Levene pode ser feito assim:

In [ ]:
from scipy import stats
stats.levene(cyl4, cylm)

O resultado, apesar de visualmente ruim, contém todos os resultados necessários: a estatística e o p-valor do teste.

Este resultado pode ser "*desempacotado*" como em uma tupla:

In [ ]:
s, p = stats.levene(cyl4, cylm)
print("Estatística do teste =", round(s, 4), "p-valor=", round(p, 5))

## Executando um teste t-Student

Embora a biblioteca *Scipy* tenha uma implementação do teste t-Student, a biblioteca *statsmodels* oferece uma funcionalidade melhor: o método `CompareMeans.from_data()` cria um objeto para comparação de médias através do modelo t-Student.

In [ ]:
import statsmodels.stats.api as sms
model = sms.CompareMeans.from_data(df[df['cylinders'] == 4]['mpg'], df[df['cylinders'] != 4]['mpg'])
model.summary( usevar='unequal')

O resultado mostra: (i) a diferença entre as médias (10,0160), o erro-padrão da diferença entre as médias (1,524), a estatística do teste (6,575), o p-valor do teste (0,000) e o intervalo de 95% de confiança para a diferença entre as médias (6,773 e 13,259).

## Análise de variáveis categóricas

Para demonstrar este conceito serão criadas 2 variáveis *dummy*.

In [ ]:
import numpy as np
df['esportivo'] = np.where((df['cylinders'] == 8) & (df['horsepower'] > 120), 1, 0)
df['leve'] = np.where(df['weight'] < 2500, 1, 0)

O primeiro passo é recodificar as variáveis *dummy* com os labels.

In [ ]:
codes = [0, 1]
lblsport = ["normal", "esportivo"]
lblweight = ["pesado", "leve"]
df['esportivo'] = df['esportivo'].replace(codes, lblsport)
df['leve'] = df['leve'].replace(codes, lblweight)
df.head()

### Criando uma tabela de contingência

A biblioteca *Pandas* possui um método bastante simples para criar tabelas de contingência (o método `crosstab()`). Tal método admite um parâmetro `margins= True` para obter os totais marginais ("*All*").

In [ ]:
vcyl_freq = pd.crosstab(
    df['esportivo'],
    df['cylinders'],
    margins = True
   )
vcyl_freq

### Calculando porcentagens por linhas

Tipicamente mostrar frequências absolutas é menos interessante que as frequências relativas. As frequências relativas são obtidas com o argumento `normalize= True`, o parâmetro `normalize= 'index'` indica que a soma deve ser por linhas (índices).

In [ ]:
vcyl_relfreq = pd.crosstab(
    df['esportivo'],
    df['cylinders'],
    margins = True,
    normalize='index'
   )
vcyl_relfreq

A tabela mostra que os motores "em linha" são mais comuns na configuração de 4 cilindros enquanto que os motores "em V" são mais comuns na configuração de 8 cilindros. Aparentemente há uma relação entre a configuração do motor e o número de cilindros.

### Testes qui-quadrado para independência

Para testar a independência entre a configuração do motor e a quantidade de cilindros é possível realizar um teste qui-quadrado.


In [ ]:
vcyl_obs = pd.crosstab(
    df['esportivo'],
    df['cylinders'],
    margins = False)
chi = stats.chi2_contingency(vcyl_obs)
chi

O resultado do método `chi2_contingency()` é uma tupla com os seguintes valores:

* A estatística do teste (`367.3441`);
* O p-valor do teste (`<0.0001`);
* Os graus de liberdade do teste (`4`);
* A matriz de frequências esperadas.

Neste caso em particular, rejeita-se a hipótese de independência entre as variáveis (como esperado).

## Correlação e diagramas de pontos

### Renomeando as colunas

Em um passo inicial, é interessante renomear as colunas para que o código fique com uma melhor estética.

In [ ]:
list(df.columns)

O método `rename()` recebe um dicionário com as relações entre os nomes atuais das colunas (chaves) e os futuros nomes (valores). O método é aplicado no eixo `columns` e altera o objeto original (`inplace= True`).

In [ ]:
df.rename(columns={'mpg': "milhas_por_galao",
                   'cylinders': 'cilindros',
                   'displacement': 'volume',
                   'horsepower': 'potencia',
                   'weight': 'peso',
                   'acceleration': 'aceleracao',
                   'model_year': 'ano_modelo',
                   'origin': 'origem',
                   'name': 'nome'}, inplace=True)
df.head()

### Diagramas de pontos

Os diagramas de pontos são utilizados para comparar a distribuição de duas variáveis numéricas (preferencialmente contínuas). É criado com o método `scatterplot()`:

In [ ]:
import seaborn as sns
sns.scatterplot(x="potencia", y="milhas_por_galao", data= df);

### Adicionando títulos

Alterar o gráfico básico da biblioteca *Seaborn* (ou *Matplotlib*) deve ser feito passo a passo. Os métodos gráficos da biblioteca *Seaborn* retorna um objeto do tipo `AxesSubplot` que pode ser manipulado.

No exemplo abaixo são adicionados um título ao gráfico e aos eixos.

In [ ]:
ax = sns.scatterplot(x="potencia", y= "milhas_por_galao", data= df)
ax.set_title("Consumo vs. potência")
ax.set_xlabel("Potência (HP)");
ax.set_ylabel("Consumo (milhas por galão)");

### Adicionando uma linha de ajuste (regressão)

O diagrama de pontos com a linha de regressão (e a banda de confiança) pode ser desenhado com o método `lmplot()` (em que "lm" significa "*linear model*" ou "*modelo linear*"):

In [ ]:
ax = sns.lmplot(x="potencia", y="milhas_por_galao", data= df)

### Adicionando cor como uma terceira dimensão

Pode ser interessante observar o efeito de uma variável categórica na reta de regressão. Isto pode ser feito através do parâmetro `hue`).

In [ ]:
sns.lmplot(x= "potencia", y= "milhas_por_galao", hue= "cilindros", data= df);

## Coeficiente de correlação

O coeficiente de correlação denota a força e o sentido da relação linear entre 2 variáveis (valores com valor absoluto próximos a `1` denotram relação forte, números positivos indicam crescimento proporcional e valores negativos indicam crescimento inversamente proporcional).

Os coeficientes de correlação (de Pearson) são calculados com o método `pearsonr()` da biblioteca *Scipy*.

In [ ]:
from scipy import stats

r, pvalue = stats.pearsonr(df['aceleracao'], df['peso'])
r, pvalue

O resultado é uma tupla com 2 valores: a correlação e o p-valor para o teste da hipótese de correlação igual a ZERO. No exemplo, a correlação é `-0.4175` indicando uma associação negativa (conforme o peso aumenta a velocidade final diminui) e esta relação pode ser considerada diferente de `0` com um nível de significância `p < 0.0001`.

### Matriz de correlação

Uma matriz de correlação é útil em diversos modelos estatísticos (exemplo: análise de agrupamento). Ela pode ser obtida com o método `corr()` do objeto *data frame*.

In [ ]:
cormat = df.corr(numeric_only= True)
round(cormat, 2)

### Exibindo a matriz de correlação como um mapa de calor

Em matrizes muito grandes é difícil obter dos números um cenário da relação entre todas as variáveis. Neste caso pode ser interessante criar um mapa de cores para as diferentes correlações. Isto se chama *mapa de calor*. O exemplo abaixo mostra a matriz de correlação para as variáveis dos dados dos automóveis.

In [ ]:
sns.heatmap(cormat, cmap='crest');